In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.5 MB/s eta 0:00:00


In [2]:
!gdown 14hIKT9eOKYliX8_nJQ4c7DwnlXjPXlRX

Downloading...
From: https://drive.google.com/uc?id=14hIKT9eOKYliX8_nJQ4c7DwnlXjPXlRX
To: /content/ds105.csv
100% 251k/251k [00:00<00:00, 8.82MB/s]


In [3]:
import pandas as pd
data = pd.read_csv('/content/ds105.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-408da3ca4a29>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
data['Price'] =  data['Price'] / 1000000
data['Price New'] =  data['Price New'] / 1000000

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1081 non-null   object 
 1   Laptop Type        1081 non-null   object 
 2   Laptop Brand       1081 non-null   object 
 3   Price              1081 non-null   float64
 4   Price New          1081 non-null   float64
 5   CPU                1081 non-null   object 
 6   CPU Brand          1081 non-null   object 
 7   Ram Size           1081 non-null   int64  
 8   Ram Type           1081 non-null   object 
 9   Hard Drive Size    1081 non-null   int64  
 10  Hard Drive         1081 non-null   object 
 11  Screen             1081 non-null   float64
 12  Resolution         1081 non-null   object 
 13  Material           1081 non-null   object 
 14  OS                 1081 non-null   object 
 15  Year               1081 non-null   int64  
 16  Graphic            1081 

# 1. Thử Nghiệm Tìm Ra Các Feature Quan Trọng

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

def cal_feature(df):
    label_encoders = {}
    for column in df.select_dtypes(include=['object']):
        label_encoder = LabelEncoder()
        df[column] = label_encoder.fit_transform(df[column])
        label_encoders[column] = label_encoder

    X = df.drop(columns=['Price'])
    y = df['Price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    return r2, mse

Tổ hợp 1: Thử nghiệm với toàn bộ biến

In [7]:
df = data.copy()
r2, mse = cal_feature(df)
print("R2 Score:", r2, "MSE Score:", mse)

R2 Score: 0.9192507002833813 MSE Score: 4.038748636471099


Tổ hợp 2: Thử nghiệm với các biến phân tích từ EDA

In [8]:
df = data[['Name', 'Laptop Type', 'Laptop Brand', 'CPU', 'CPU Brand','Ram Type',
             'Hard Drive', 'Resolution', 'OS', 'Graphic',
             'Graphic Name', 'Price New', 'Ram Size', 'Hard Drive Size', "Price"]].copy()
r2, mse = cal_feature(df)
print("R2 Score:", r2, "MSE Score:", mse)

R2 Score: 0.9046761838157408 MSE Score: 4.767706147155177


Tổ hợp 3: * + Warranty Time, Warranty Location



In [9]:
df = data[['Name', 'Laptop Type', 'Laptop Brand', 'CPU', 'CPU Brand','Ram Type',
          'Hard Drive', 'Resolution', 'OS', 'Graphic', 'Graphic Name', 'Price',
          'Price New', 'Ram Size', 'Hard Drive Size', 'Warranty Time',
          'Warranty Location']].copy()
r2, mse = cal_feature(df)
print("R2 Score:", r2, "MSE Score:", mse)

R2 Score: 0.913749295607676 MSE Score: 4.313906324657309


Tổ hợp 4: * + Warranty Time, Year, Warranty Location


In [10]:
df = data[['Name', 'Laptop Type', 'Laptop Brand', 'CPU', 'CPU Brand','Ram Type',
          'Hard Drive', 'Resolution', 'OS', 'Graphic', 'Graphic Name', 'Price',
          'Price New', 'Ram Size', 'Hard Drive Size', 'Warranty Time', 'Year',
          'Warranty Location']].copy()
r2, mse = cal_feature(df)
print("R2 Score:", r2, "MSE Score:", mse)

R2 Score: 0.9242760911994939 MSE Score: 3.787399203331592


**Kết Luận:** Có thể thấy với 4 tổ hợp dựa vào kết quả R2 Socre và MSE Score có thể chọn ra tổ hợp 4 để phát triển mô hình gồm 17 biến thành phần.

# 2. Thử Nghiệm Mô Hình

In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
#CPU cores
n_jobs = cpu_count()

In [12]:
df = data[['Name', 'Laptop Type', 'Laptop Brand', 'CPU', 'CPU Brand','Ram Type',
          'Hard Drive', 'Resolution', 'OS', 'Graphic', 'Graphic Name', 'Price',
          'Price New', 'Ram Size', 'Hard Drive Size', 'Warranty Time', 'Year',
          'Warranty Location']].copy()

In [13]:
label_encoders = {}
for column in df.select_dtypes(include=['object']):
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
    label_encoders[column] = label_encoder

X = df.drop(columns=['Price'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train

,Name,Laptop Type,Laptop Brand,CPU,CPU Brand,Ram Type,Hard Drive,Resolution,OS,Graphic,Graphic Name,Price New,Ram Size,Hard Drive Size,Warranty Time,Year,Warranty Location
628,198,1,8,43,2,5,1,1,3,1,10,26.99,16,512,22.5,2022,0
644,83,1,2,51,2,3,1,1,3,0,25,33.99,16,512,16.5,2022,0
884,58,0,2,52,2,4,1,5,3,0,29,32.99,16,512,21.5,2022,0
591,172,1,8,30,2,3,1,5,3,1,10,19.49,8,512,28.0,2022,0
342,171,1,8,30,2,3,1,5,3,1,10,18.99,8,512,34.0,2022,0
853,116,1,6,30,2,3,1,5,3,1,11,18.59,8,512,11.0,2022,0
312,0,1,0,18,2,3,1,7,3,1,11,6.99,4,256,10.0,2021,0
308,0,1,0,18,2,3,1,7,3,1,11,6.99,4,256,7.0,2021,0
736,197,1,8,63,2,6,1,14,3,1,10,34.99,16,1024,33.0,2022,0
237,90,1,2,31,2,7,1,1,3,1,10,25.49,8,512,17.5,2022,0


## Save Label_Encoder

In [15]:
import os
import pickle

model_dir = '/content/model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, 'label_encoders.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(label_encoders, file)

In [16]:
#Decoder
# import pandas as pd
# import pickle

# with open('label_encoders.pkl', 'rb') as file:
#     label_encoders = pickle.load(file)

# for column in df.select_dtypes(include=['object']):
#     label_encoder = label_encoders[column]
#     df[column] = label_encoder.transform(df[column])

# print(df)

## Linear Regression

### Normal

In [17]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 3.787399203331574
R2 on test set: 0.9242760911994944
Average MSE on 5-fold cross-validation: 3.9138036042867164
Average R2 on 5-fold cross-validation: 0.9009492407681485
STD of MSE on 5-fold cross-validation: 0.43867710597608367
STD of R2 on 5-fold cross-validation: 0.013247514689039027


In [18]:
model_path = os.path.join(model_dir, 'linear.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(pipeline, file)

###  Polynomial Features

In [19]:
for i in range(1,4):
    print(f"--- Polynomial Features Degree = {i} ---")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree = i)),
        ('regressor', LinearRegression())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE on test set: {mse}")
    print(f"R2 on test set: {r2}")

    k = 5
    scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
    avg_mse = -scores.mean()
    avg_r2 = r2.mean()
    std_mse = np.std(-scores)
    std_r2 = np.std(r2)
    print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
    print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
    print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
    print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

--- Polynomial Features Degree = 1 ---
MSE on test set: 3.7873992033315638
R2 on test set: 0.9242760911994945
Average MSE on 5-fold cross-validation: 3.913803604286721
Average R2 on 5-fold cross-validation: 0.9009492407681481
STD of MSE on 5-fold cross-validation: 0.4386771059760789
STD of R2 on 5-fold cross-validation: 0.013247514689038902
--- Polynomial Features Degree = 2 ---
MSE on test set: 2.2744926484706837e+23
R2 on test set: -4.5475394758680306e+21
Average MSE on 5-fold cross-validation: 2.6509099349309204e+16
Average R2 on 5-fold cross-validation: -720741490297798.9
STD of MSE on 5-fold cross-validation: 5.30181986986184e+16
STD of R2 on 5-fold cross-validation: 1441482980595599.2
--- Polynomial Features Degree = 3 ---
MSE on test set: 6.313199900428138e+22
R2 on test set: -1.26223867048094e+21
Average MSE on 5-fold cross-validation: 8.819735544763996e+18
Average R2 on 5-fold cross-validation: -2.1860648723580845e+17
STD of MSE on 5-fold cross-validation: 9.109861198097893e+1

## Lasso

### Normal

In [20]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 8.676412948306348
R2 on test set: 0.826526894171827
Average MSE on 5-fold cross-validation: 7.028461265576496
Average R2 on 5-fold cross-validation: 0.8214028720512039
STD of MSE on 5-fold cross-validation: 0.9415270589102677
STD of R2 on 5-fold cross-validation: 0.031646442996957555


###  Polynomial Features

In [21]:
for i in range(1,4):
    print(f"--- Polynomial Features Degree = {i} ---")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree = i)),
        ('regressor', Lasso())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE on test set: {mse}")
    print(f"R2 on test set: {r2}")

    k = 5
    scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
    avg_mse = -scores.mean()
    avg_r2 = r2.mean()
    std_mse = np.std(-scores)
    std_r2 = np.std(r2)
    print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
    print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
    print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
    print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

--- Polynomial Features Degree = 1 ---
MSE on test set: 8.676412948306348
R2 on test set: 0.826526894171827
Average MSE on 5-fold cross-validation: 7.028461265576496
Average R2 on 5-fold cross-validation: 0.8214028720512039
STD of MSE on 5-fold cross-validation: 0.9415270589102677
STD of R2 on 5-fold cross-validation: 0.031646442996957555
--- Polynomial Features Degree = 2 ---
MSE on test set: 7.430176551419414
R2 on test set: 0.8514437001897204
Average MSE on 5-fold cross-validation: 6.1767955397354495
Average R2 on 5-fold cross-validation: 0.843412975695291
STD of MSE on 5-fold cross-validation: 0.56774498821486
STD of R2 on 5-fold cross-validation: 0.02058400412445588
--- Polynomial Features Degree = 3 ---
MSE on test set: 6.235566157361126
R2 on test set: 0.8753283143207665
Average MSE on 5-fold cross-validation: 6.109185409705392
Average R2 on 5-fold cross-validation: 0.8459816330497884
STD of MSE on 5-fold cross-validation: 0.459486048342927
STD of R2 on 5-fold cross-validation: 

### Optuna

In [22]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.0, 1.0)
    degree = trial.suggest_categorical('degree', [1, 2, 3])

    X_train_subset, X_val_subset, y_train_subset, y_val_subset = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=degree)),
        ('regressor', Lasso(alpha=alpha))
    ])

    pipeline.fit(X_train_subset, y_train_subset)
    y_pred = pipeline.predict(X_val_subset)
    mse = mean_squared_error(y_val_subset, y_pred)

    return mse

In [23]:
import optuna.logging as logging

optuna.logging.disable_default_handler()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_mse = study.best_value
print("Best parameters: ", best_params)
print("Best MSE: ", best_mse)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.132e+00, tolerance: 2.805e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.999e+00, tolerance: 2.805e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.324e+01, tolerance: 2.805e

Best parameters:  {'alpha': 0.046749688433424696, 'degree': 2}
Best MSE:  2.595311847692224


In [24]:
best_alpha = best_params['alpha']
best_degree = best_params['degree']

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree = best_degree)),
    ('regressor', Lasso(alpha=best_alpha))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 2.4081110243121433
R2 on test set: 0.9518530870930901
Average MSE on 5-fold cross-validation: 2.3052875292602244
Average R2 on 5-fold cross-validation: 0.9414748751348823
STD of MSE on 5-fold cross-validation: 0.7609087363772558
STD of R2 on 5-fold cross-validation: 0.019948681873604306


In [25]:
model_path = os.path.join(model_dir, 'lasso.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(pipeline, file)

## Ridge

### Normal

In [26]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 3.803681508977274
R2 on test set: 0.9239505485879068
Average MSE on 5-fold cross-validation: 3.913461891560489
Average R2 on 5-fold cross-validation: 0.9009557431645574
STD of MSE on 5-fold cross-validation: 0.43169433506061794
STD of R2 on 5-fold cross-validation: 0.013108960390585168


###  Polynomial Features

In [27]:
for i in range(1,4):
    print(f"--- Polynomial Features Degree = {i} ---")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree = i)),
        ('regressor', Ridge())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE on test set: {mse}")
    print(f"R2 on test set: {r2}")

    k = 5
    scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
    avg_mse = -scores.mean()
    avg_r2 = r2.mean()
    std_mse = np.std(-scores)
    std_r2 = np.std(r2)
    print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
    print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
    print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
    print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

--- Polynomial Features Degree = 1 ---
MSE on test set: 3.803681508977279
R2 on test set: 0.9239505485879067
Average MSE on 5-fold cross-validation: 3.913461891560483
Average R2 on 5-fold cross-validation: 0.9009557431645575
STD of MSE on 5-fold cross-validation: 0.43169433506061733
STD of R2 on 5-fold cross-validation: 0.013108960390585157
--- Polynomial Features Degree = 2 ---
MSE on test set: 3.748744195207167
R2 on test set: 0.9250489456446557
Average MSE on 5-fold cross-validation: 2.927451261219208
Average R2 on 5-fold cross-validation: 0.9259827842536265
STD of MSE on 5-fold cross-validation: 1.00902594462362
STD of R2 on 5-fold cross-validation: 0.026948197576101545
--- Polynomial Features Degree = 3 ---
MSE on test set: 32.87074962284724
R2 on test set: 0.34279395621797415
Average MSE on 5-fold cross-validation: 15.798907108468331
Average R2 on 5-fold cross-validation: 0.610844303758962
STD of MSE on 5-fold cross-validation: 8.090081111377613
STD of R2 on 5-fold cross-validati

### Optuna

In [28]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.0, 1.0)
    degree = trial.suggest_categorical('degree', [1, 2, 3])

    X_train_subset, X_val_subset, y_train_subset, y_val_subset = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=degree)),
        ('regressor', Ridge(alpha=alpha))
    ])

    pipeline.fit(X_train_subset, y_train_subset)
    y_pred = pipeline.predict(X_val_subset)
    mse = mean_squared_error(y_val_subset, y_pred)

    return mse

In [29]:
import optuna.logging as logging

optuna.logging.disable_default_handler()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_mse = study.best_value
print("Best parameters: ", best_params)
print("Best MSE: ", best_mse)

Best parameters:  {'alpha': 0.9998976143475337, 'degree': 2}
Best MSE:  4.029042552082765


In [30]:
best_alpha = best_params['alpha']
best_degree = best_params['degree']

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree = best_degree)),
    ('regressor', Ridge(alpha=best_alpha))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 3.7487806008862585
R2 on test set: 0.9250482177624929
Average MSE on 5-fold cross-validation: 2.9275038012785153
Average R2 on 5-fold cross-validation: 0.9259814608975578
STD of MSE on 5-fold cross-validation: 1.0090808711486303
STD of R2 on 5-fold cross-validation: 0.026949614920661245


In [31]:
model_path = os.path.join(model_dir, 'ridge.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(pipeline, file)

## ElasticNet

### Normal

In [32]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', ElasticNet())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 12.168038385592995
R2 on test set: 0.7567165805544925
Average MSE on 5-fold cross-validation: 8.879185557141307
Average R2 on 5-fold cross-validation: 0.7745791750064207
STD of MSE on 5-fold cross-validation: 0.9896679279661839
STD of R2 on 5-fold cross-validation: 0.03562741774283429


###  Polynomial Features

In [33]:
for i in range(1,4):
    print(f"--- Polynomial Features Degree = {i} ---")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree = i)),
        ('regressor', ElasticNet())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE on test set: {mse}")
    print(f"R2 on test set: {r2}")

    k = 5
    scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
    avg_mse = -scores.mean()
    avg_r2 = r2.mean()
    std_mse = np.std(-scores)
    std_r2 = np.std(r2)
    print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
    print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
    print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
    print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

--- Polynomial Features Degree = 1 ---
MSE on test set: 12.168038385592995
R2 on test set: 0.7567165805544925
Average MSE on 5-fold cross-validation: 8.879185557141307
Average R2 on 5-fold cross-validation: 0.7745791750064207
STD of MSE on 5-fold cross-validation: 0.9896679279661843
STD of R2 on 5-fold cross-validation: 0.03562741774283429
--- Polynomial Features Degree = 2 ---
MSE on test set: 10.216547646047145
R2 on test set: 0.7957339903528607
Average MSE on 5-fold cross-validation: 7.352160553246859
Average R2 on 5-fold cross-validation: 0.8140079520566037
STD of MSE on 5-fold cross-validation: 0.4491426985388786
STD of R2 on 5-fold cross-validation: 0.0159914448734216
--- Polynomial Features Degree = 3 ---
MSE on test set: 5.759424329401374
R2 on test set: 0.8848481242010702
Average MSE on 5-fold cross-validation: 8.634908089230034
Average R2 on 5-fold cross-validation: 0.785201640404404
STD of MSE on 5-fold cross-validation: 3.6037298839433016
STD of R2 on 5-fold cross-validatio

### Optuna

In [34]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.0, 1.0)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    degree = trial.suggest_categorical('degree', [1, 2, 3])

    X_train_subset, X_val_subset, y_train_subset, y_val_subset = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=degree)),
        ('regressor', ElasticNet(alpha=alpha, l1_ratio=l1_ratio))
    ])

    pipeline.fit(X_train_subset, y_train_subset)
    y_pred = pipeline.predict(X_val_subset)
    mse = mean_squared_error(y_val_subset, y_pred)

    return mse

In [35]:
import optuna.logging as logging

optuna.logging.disable_default_handler()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_mse = study.best_value
print("Best parameters: ", best_params)
print("Best MSE: ", best_mse)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.638e+01, tolerance: 2.805e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.785e+01, tolerance: 2.805e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.759e+01, tolerance: 2.805e

Best parameters:  {'alpha': 0.0541153921294669, 'l1_ratio': 0.0009892834347008592, 'degree': 2}
Best MSE:  2.5284832360771414


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.547e+02, tolerance: 2.805e+00
  model = cd_fast.enet_coordinate_descent(


In [36]:
best_alpha = best_params['alpha']
best_degree = best_params['degree']
best_l1_ratio = best_params['l1_ratio']

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree = best_degree)),
    ('regressor', ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.699e+00, tolerance: 3.446e+00
  model = cd_fast.enet_coordinate_descent(


MSE on test set: 3.2288465924708016
R2 on test set: 0.9354435928792486
Average MSE on 5-fold cross-validation: 2.6015063981731723
Average R2 on 5-fold cross-validation: 0.933676208661344
STD of MSE on 5-fold cross-validation: 0.914390674153831
STD of R2 on 5-fold cross-validation: 0.02520189851591964


In [37]:
model_path = os.path.join(model_dir, 'elasticnet.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(pipeline, file)

## RandomForestRegressor

### Normal

In [38]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 3.226702179665795
R2 on test set: 0.9354864675040133
Average MSE on 5-fold cross-validation: 1.3596926987741762
Average R2 on 5-fold cross-validation: 0.9661971681862056
STD of MSE on 5-fold cross-validation: 0.5616684070837131
STD of R2 on 5-fold cross-validation: 0.01364286788455986


###  Polynomial Features

In [39]:
for i in range(1,4):
    print(f"--- Polynomial Features Degree = {i} ---")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree = i)),
        ('regressor', RandomForestRegressor())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE on test set: {mse}")
    print(f"R2 on test set: {r2}")

    k = 5
    scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
    avg_mse = -scores.mean()
    avg_r2 = r2.mean()
    std_mse = np.std(-scores)
    std_r2 = np.std(r2)
    print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
    print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
    print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
    print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

--- Polynomial Features Degree = 1 ---
MSE on test set: 3.3059756420017385
R2 on test set: 0.9339015021729369
Average MSE on 5-fold cross-validation: 1.345280430104649
Average R2 on 5-fold cross-validation: 0.9673887263404944
STD of MSE on 5-fold cross-validation: 0.5685156105648893
STD of R2 on 5-fold cross-validation: 0.011313100943661865
--- Polynomial Features Degree = 2 ---
MSE on test set: 3.1974347173679565
R2 on test set: 0.9360716307062219
Average MSE on 5-fold cross-validation: 1.7291208382254317
Average R2 on 5-fold cross-validation: 0.9578535473443639
STD of MSE on 5-fold cross-validation: 0.9402632100473901
STD of R2 on 5-fold cross-validation: 0.021185933971824823
--- Polynomial Features Degree = 3 ---
MSE on test set: 2.730464025188229
R2 on test set: 0.9454080761688554
Average MSE on 5-fold cross-validation: 1.7633313517038594
Average R2 on 5-fold cross-validation: 0.9539620497549335
STD of MSE on 5-fold cross-validation: 0.7499460475350845
STD of R2 on 5-fold cross-val

### Optuna

In [40]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    degree = trial.suggest_categorical('degree', [1, 2])

    X_train_subset, X_val_subset, y_train_subset, y_val_subset = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=degree)),
        ('regressor', RandomForestRegressor(n_estimators=n_estimators,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  random_state=42))
    ])

    pipeline.fit(X_train_subset, y_train_subset)
    y_pred = pipeline.predict(X_val_subset)
    mse = mean_squared_error(y_val_subset, y_pred)

    return mse

In [41]:
import optuna.logging as logging
logging.disable_default_handler()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=80)

best_params = study.best_params
best_mse = study.best_value
print("Best parameters: ", best_params)
print("Best MSE: ", best_mse)

Best parameters:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'degree': 2}
Best MSE:  1.4346846079768802


In [42]:
best_degree = best_params['degree']
best_n_estimators = best_params['n_estimators']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree = best_degree)),
    ('regressor', RandomForestRegressor(n_estimators=best_n_estimators,
                                  min_samples_split=best_min_samples_split,
                                  min_samples_leaf=best_min_samples_leaf,
                                  random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE on test set: {mse}")
print(f"R2 on test set: {r2}")

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=n_jobs, scoring='r2')
avg_mse = -scores.mean()
avg_r2 = r2.mean()
std_mse = np.std(-scores)
std_r2 = np.std(r2)
print(f"Average MSE on {k}-fold cross-validation: {avg_mse}")
print(f"Average R2 on {k}-fold cross-validation: {avg_r2}")
print(f"STD of MSE on {k}-fold cross-validation: {std_mse}")
print(f"STD of R2 on {k}-fold cross-validation: {std_r2}")

MSE on test set: 2.882260624092431
R2 on test set: 0.9423731091124286
Average MSE on 5-fold cross-validation: 1.6451406218372988
Average R2 on 5-fold cross-validation: 0.9586103849276413
STD of MSE on 5-fold cross-validation: 0.805712461838193
STD of R2 on 5-fold cross-validation: 0.02018241001145193


In [43]:
model_path = os.path.join(model_dir, 'randomforest.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(pipeline, file)

In [44]:
import joblib
joblib.dump(pipeline, "pipeline.joblib")

['pipeline.joblib']